In [ ]:
pip install -q -U google-generativeai

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import google.generativeai as genai

In [2]:
path = input("Enter The Path of The Html File")
with open(path, 'r', encoding = 'utf8') as f:
    contents = f.read()

Enter The Path of The Html File D:\C\Linkedin Project\linkedin2.html


In [3]:
def full_link(links):
    for i in links:
        i = i[:i.find("?")]
        sliced_links.append(i) 
    return links

def job_detail(s):
    s_l = []
    for span in s:
        s_l.append(span.text.strip())
    return s_l

def company_web(l):
    return l[:l.find("?")] 

def slec_task(s):
    
    s_l = []
    for strong in s:
        s_l.append(strong.text.strip())
    genai.configure(api_key='--------')

    model = genai.GenerativeModel('gemini-pro')

    response2 = model.generate_content(str(s_l) + 'summarize it in 20 words')
    
    return response2.text


In [4]:
def scraper(raw_data):
    
    soup1 = BeautifulSoup(contents, 'html.parser')
    n = int(input('Enter the number of Jobs to Scrape'))
    
    print('<===========================================Starting...... Just a Few Arrangemnts===========================================>')
    links = []
    sliced_links = []
    job_title = []
    job_details = []
    company_website = []
    selection_task = []
    
    link_soup = soup1.find_all('a', attrs = {'class':"base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]",'href':True})

    for tag in link_soup:
        links.append(tag['href'])
    
    for i in links:
        i = i[:i.find("?")]
        sliced_links.append(i)
    
    print('<=============================================Started Scraping=============================================>')
    
    for i in sliced_links[:n]:
        response = requests.get(i)
        print("Reponse Got "+ i)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            print("staus ok")
            
            if(soup.find_all('figcaption', attrs = {'class': 'closed-job__flavor--closed'})):
                continue
            else:    

                try:
                    job_title.append(soup.find('h1',attrs = {'class':'top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title'}).text)
                except Exception as e:
                    job_title.append(f"An unexpected error occurred: {e}")
                try:    
                    company_website.append(company_web(soup.find('a', attrs = {'class': 'topcard__org-name-link topcard__flavor--black-link', 'href':True}).get('href')))
                except Exception as e:
                    company_website.append(f"An unexpected error occurred: {e}")
                try:
                    job_details.append(job_detail(soup.find_all('span', attrs = {'class': 'description__job-criteria-text description__job-criteria-text--criteria'})))
                except Exception as e:
                    job_details.append(f"An unexpected error occurred: {e}")
                try:    
                    selection_task.append(slec_task(soup.find('div', attrs = {'class': 'show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden'})))
                except Exception as e:
                    selection_task.append(f"An unexpected error occurred: {e}")

        else:
            print('Status NOT OK')
            
    print('appending data')            

    seniority_level, employment_type, job_function, industry_type = zip(*job_details)
    data = { 'Job_Title' : job_title,
             'Company_Website' : company_website,
             'Seniority Level': seniority_level,
             'Employment Type': employment_type,
             'Job Function': job_function,
             'Industry Type': industry_type,
             'Selected Role Daily Task' : selection_task
    
    }        
    print('<================================Creating DataFrame==================================>')
    print('<================================Check Your Directory================================>')
    return pd.DataFrame(data)


In [6]:
name = input('Enter the Name you want to give to the csv file')+'.csv'
name

Enter the Name you want to give to the csv file final


'final.csv'

In [7]:
%%time
scraper(contents).to_csv(name)

Enter the number of Jobs to Scrape 500


<===========================================Starting...... Just a Few Arrangemnts===========================================>
<=============================================Started Scraping=============================================>
Reponse Got https://in.linkedin.com/jobs/view/data-scientist-at-fusemachines-3809364262
staus ok
Reponse Got https://in.linkedin.com/jobs/view/ai-ml-developer-at-nexus-hire-3817004185
Status NOT OK
Reponse Got https://in.linkedin.com/jobs/view/data-scientist-at-acme-services-3815717557
staus ok
Reponse Got https://in.linkedin.com/jobs/view/data-scientist-kochi-at-ripplesindia-group-3817311701
staus ok
Reponse Got https://in.linkedin.com/jobs/view/software-engineer-ai-ml-at-mindfire-solutions-3817032400
staus ok
Reponse Got https://in.linkedin.com/jobs/view/data-scientist-at-onebanc-3816857664
staus ok
Reponse Got https://in.linkedin.com/jobs/view/data-science-at-acme-services-3816940402
staus ok
Reponse Got https://in.linkedin.com/jobs/view/data-scientist